In [20]:
import numpy as np
import cv2
import math
from skimage.transform import hough_circle, hough_circle_peaks
import matplotlib.pyplot as plt
from skimage.filters.rank import equalize
from skimage.morphology import disk
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
import datetime

In [21]:

def IrisLocalization(eye): # input is the eye image in grayscale
    blured = cv2.bilateralFilter(eye, 9, 100, 100) # remove noise
    Xp = blured.sum(axis=0).argmin() # index of the column with the least sum
    Yp = blured.sum(axis=1).argmin() # index of the row with the least sum
    x = blured[max(Yp - 60, 0):min(Yp + 60, 280), max(Xp - 60, 0):min(Xp + 60, 320)].sum(axis=0).argmin()
    y = blured[max(Yp - 60, 0):min(Yp + 60, 280), max(Xp - 60, 0):min(Xp + 60, 320)].sum(axis=1).argmin()
    Xp = max(Xp - 60, 0) + x
    Yp = max(Yp - 60, 0) + y
    if Xp >= 100 and Yp >= 80: # check if the pupil center is not too close to the edge
        blur = cv2.GaussianBlur(eye[Yp - 60:Yp + 60, Xp - 60:Xp + 60], (5, 5), 0)
        pupil_circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=200, param1=200, param2=12, minRadius=15, maxRadius=80)
        xp, yp, rp = np.round(pupil_circles[0][0]).astype("int")
        xp = Xp - 60 + xp
        yp = Yp - 60 + yp
    else:
        pupil_circles = cv2.HoughCircles(blured, cv2.HOUGH_GRADIENT, 4, 280, minRadius=25, maxRadius=55, param2=51)
        xp, yp, rp = np.round(pupil_circles[0][0]).astype("int")

    # Draw the circle on the original image
    # fig, ax = plt.subplots()
    # ax.imshow(eye, cmap='gray')  # Display the grayscale image
    # pupil_circle = plt.Circle((xp, yp), rp, color='red', fill=False, linewidth=2)  # Red circle
    # ax.add_patch(pupil_circle)
    # ax.plot(xp, yp, 'bo')  # Blue point at the center
    # plt.title("Detected Pupil Circle")
    # plt.axis('off')  # Turn off axes for better visualization
    # plt.show()

    eye_copy = eye.copy()
    rp = rp + 7 # slightly enlarge the pupil radius makes a better result
    blured_copy = cv2.medianBlur(eye_copy, 11)
    blured_copy = cv2.medianBlur(blured_copy, 11)
    blured_copy = cv2.medianBlur(blured_copy, 11)
    eye_edges = cv2.Canny(blured_copy, threshold1=15, threshold2=30, L2gradient=True) # edge detection

    # Display the edge-detected image with pupil edges
    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.title("With Pupil Edges")
    # plt.imshow(eye_edges, cmap='gray')

    # Remove the edge of the pupil
    eye_edges[:, xp - rp - 30:xp + rp + 30] = 0

    # Display the edge-detected image without pupil edges
    # plt.subplot(1, 2, 2)
    # plt.title("Without Pupil Edges")
    # plt.imshow(eye_edges, cmap='gray')
    # plt.show()

    hough_radii = np.arange(rp+45, 150, 2) # possible iris radii
    hough_res = hough_circle(eye_edges, hough_radii) # hough transform to detect circles with different radii
    accums, xi, yi, ri = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1) # find the mostt prominent circle (peak) to be the iris circle
    iris = [] # iris circle
    iris.extend(xi)
    iris.extend(yi)
    iris.extend(ri)
    if ((iris[0] - xp) ** 2+(iris[1]-yp)**2) ** 0.5 > rp* 0.3: # using Euclidean distance to check if the iris circle is too far from the pupil, use the pupil circle instead
        iris[0] = xp
        iris[1] = yp
    
    # fig, ax = plt.subplots()
    # ax.imshow(eye, cmap='gray')  # Display the grayscale image
    # Draw the blue circle for the iris
    # iris_circle = plt.Circle((iris[0], iris[1]), iris[2], color='blue', fill=False, linewidth=2)
    # ax.add_patch(iris_circle)
    # ax.plot(iris[0], iris[1], 'bo')  # Blue point at the iris center
    # # Configure the plot
    # plt.title("Without Pupil Edges")
    # plt.axis('off')  # Turn off axes for better visualization
    # plt.show()

    return np.array(iris), np.array([xp,yp,rp]) # return the iris and pupil circles

In [22]:
def IrisNormalization(image,inner_circle,outer_circle ):
    localized_img=image
    row=64
    col=512
    normalized_iris=np.zeros(shape=(64,512))
    inner_y=inner_circle[0]  #height
    inner_x=inner_circle[1] #width
    outer_y=outer_circle[0]
    outer_x=outer_circle[1]
    angle=2.0*math.pi/col
    # 1 row 512 col
    inner_boundary_x = np.zeros(shape=(1,col)) # x coordinate of pupil boundary for each angle
    inner_boundary_y = np.zeros(shape=(1,col)) # y coordinate of pupil boundary for each angle
    outer_boundary_x = np.zeros(shape=(1,col)) # x coordinate of iris boundary for each angle
    outer_boundary_y = np.zeros(shape=(1,col)) # y coordinate of iris boundary for each angle
    for j in range(col):


        inner_boundary_x[0][j]=inner_circle[0]+inner_circle[2]*math.cos(angle*(j))
        inner_boundary_y[0][j]=inner_circle[1]+inner_circle[2]*math.sin(angle*(j))
        
        outer_boundary_x[0][j]=outer_circle[0]+outer_circle[2]*math.cos(angle*(j))
        outer_boundary_y[0][j]=outer_circle[1]+outer_circle[2]*math.sin(angle*(j))
        
    for j in range (512):
        for i in range (64):
            normalized_iris[i][j]=localized_img[
                min(int(int(inner_boundary_y[0][j])+(int(outer_boundary_y[0][j])-int(inner_boundary_y[0][j]))*(i/64.0)),localized_img.shape[0]-1)
            ][
                min(int(int(inner_boundary_x[0][j])+(int(outer_boundary_x[0][j])-int(inner_boundary_x[0][j]))*(i/64.0)),localized_img.shape[1]-1)
            ]

    res_image=255-normalized_iris

    # plt.imshow(normalized_iris, cmap='gray')
    # plt.title("Normalized Iris")
    # plt.show()
    return res_image

In [23]:
def ImageEnhancement(normalized_iris):
    row=64
    col=512
    normalized_iris = normalized_iris.astype(np.uint8)
    enhanced_image=normalized_iris
    enhanced_image = equalize(enhanced_image, disk(32))
    roi = enhanced_image[0:48,:]
    plt.imshow(roi, cmap='gray')
    plt.title("Enhanced Iris")
    plt.show()
    return roi

In [24]:
def gabor_filter_bank(scales, orientations, ksize=31, sigma=2.24):
    """
    Generates a bank of Gabor filters.
    :param scales: Number of scales.
    :param orientations: Number of orientations.
    :param ksize: Size of the Gabor filter kernel.
    :param sigma: Standard deviation of the Gaussian envelope.
    :return: List of Gabor filters.
    """
    filters = []
    for scale in range(scales):
        for orientation in range(orientations):
            theta = orientation * np.pi / orientations
            lambd = 4
            gamma = 1
            kernel_real = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi=0, ktype=cv2.CV_64F)
            kernel_imag = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi=np.pi/2, ktype=cv2.CV_64F)
            filters.append((kernel_real, kernel_imag))
    
    # Access the first Gabor filter (real and imaginary parts)
    kernel_real = filters[0][0]
    kernel_imag = filters[0][1]

    # Visualize the first Gabor filter (real and imaginary parts)
    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(10, 5))

    # plt.subplot(1, 2, 1)
    # plt.imshow(kernel_real, cmap='gray')
    # plt.title('Real Part of First Gabor Filter')
    # plt.colorbar()

    # plt.subplot(1, 2, 2)
    # plt.imshow(kernel_imag, cmap='gray')
    # plt.title('Imaginary Part of First Gabor Filter')
    # plt.colorbar()

    # plt.show()
    return filters

def apply_gabor_filters(image, filters):
    """
    Apply a bank of Gabor filters to an image.
    :param image: Input normalized iris image.
    :param filters: List of Gabor filters.
    :return: List of filtered responses (real and imaginary).
    """
    responses = []
    for kernel_real, kernel_imag in filters:
        # Apply the real part of the filter
        response_real = cv2.filter2D(image, cv2.CV_64F, kernel_real)
        # Apply the imaginary part of the filter
        response_imag = cv2.filter2D(image, cv2.CV_64F, kernel_imag)
        # Compute the magnitude of the response
        magnitude = np.sqrt(response_real**2 + response_imag**2)
        responses.append((response_real, response_imag, magnitude))
    return responses

def phase_quantization(responses):
    """
    Quantize the phase of Gabor filter responses into 2-bit binary codes.
    :param responses: List of Gabor filter responses (complex values).
    :return: Binary feature vector (iris code).
    """
    binary_code = []
    for response in responses:
        real = response[0]
        imag = response[1]
        phase = np.arctan2(imag, real)
        quantized_phase = ((phase >= 0) & (phase < np.pi/2)) * 0b00 + \
                          ((phase >= np.pi/2) & (phase < np.pi)) * 0b01 + \
                          ((phase >= -np.pi) & (phase < -np.pi/2)) * 0b10 + \
                          ((phase >= -np.pi/2) & (phase < 0)) * 0b11
        
        # Convert each quantized value to 2 bits (binary representation)
        binary_block = np.unpackbits(np.uint8(quantized_phase).reshape(-1, 1), axis=1)[:, -2:]
        binary_code.append(binary_block.flatten())
  
    return np.hstack(binary_code)

def FeatureExtraction(image, block_size, scales=1, orientations=4):
    """
    Generate the binary feature vector (iris code) for an input image.
    :param image: Normalized iris image.
    :param block_size: Size of the blocks for pixel averaging.
    :param scales: Number of scales for Gabor filters.
    :param orientations: Number of orientations for Gabor filters.
    :return: Binary feature vector (iris code).
    """
    # Step 1: Divide the image into blocks and calculate block mean
    h, w = image.shape
    h_blocks, w_blocks = h // block_size, w // block_size
    block_means = np.zeros((h_blocks, w_blocks))
    for i in range(h_blocks):
        for j in range(w_blocks):
            block = image[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size]
            block_means[i, j] = np.mean(block)

    # Step 2: Generate Gabor filter bank
    filters = gabor_filter_bank(scales=scales, orientations=orientations)
    # print("First filter: ", filters[0]) 

    # Step 3: Apply Gabor filters to the block means
    responses = apply_gabor_filters(block_means, filters)

    # Step 4: Quantize phase to generate binary iris code
    iris_code = phase_quantization(responses)

    return iris_code


In [25]:
def circular_shift(code, shift):
    """
    Circularly shift a binary code (list of 0s and 1s).
    
    Args:
        code: List of binary values (e.g., [1, 0, 1, 0, 1]).
        shift: Number of positions to shift. Positive for left, negative for right.
    
    Returns:
        Shifted binary code.
    """
    n = len(code)
    shift = shift % n  # Handle shifts greater than the length of the code
    return code[shift:] + code[:shift]

def HammingDistance(code1, code2, mask1=None, mask2=None):
    """
    Compute the Hamming distance between two iris codes.
    :param code1: First binary iris code (numpy array).
    :param code2: Second binary iris code (numpy array).
    :param mask1: Mask for code1 (optional).
    :param mask2: Mask for code2 (optional).
    :return: Hamming distance.
    """
    if mask1 is not None and mask2 is not None:
        # Combine masks and exclude masked bits
        combined_mask = mask1 & mask2
        differing_bits = np.sum((code1 ^ code2) & ~combined_mask)
        total_bits = np.sum(~combined_mask)
    else:
        # Without masks
        differing_bits = np.sum(code1 ^ code2)
        total_bits = code1.size
    
    if total_bits == 0:
        return float('inf')  # Handle case where no bits are available for comparison
    return differing_bits / total_bits

def find_min_hamming_distance(reference_code, query_code):
    """
    Find the minimum Hamming distance by circularly shifting the query code.
    
    Args:
        reference_code: The reference binary code (list of 0s and 1s).
        query_code: The query binary code (list of 0s and 1s).
    
    Returns:
        Tuple containing the minimum Hamming distance and the optimal shift value.
    """
    n = len(query_code)
    min_distance = float('inf')
    best_shift = 0
    
    for shift in range(n):
        shifted_code = circular_shift(query_code, shift)
        reference_code = np.array(reference_code)
        shifted_code = np.array(shifted_code)
        distance = HammingDistance(reference_code, shifted_code)
        if distance < min_distance:
            min_distance = distance
            best_shift = shift
    
    return min_distance, best_shift

In [ ]:
# Example usage
# Define the root path to the dataset
rootpath = "D:/at school/2024.1/Biometric Authentication System/Project/Biometric/IrisRecognition-master/CASIA Iris Image Database (version 1.0)/"

# Record the start time of the process
starttime = datetime.datetime.now()

# Loop through each subject in the dataset
med = 0
for i in range(1,3):
    # Define paths for training and testing images for the current subject
    filespath = rootpath + str(i).zfill(3)
    trainpath = filespath + "/1/"
    testpath = filespath + "/2/"

    train_iris_code = np.zeros((4, 1024))
    test_iris_code = np.zeros((5, 1024))

    # Loop through each training image for the current subject
    for j in range(1,4):
        # Construct the file path for the current training image
        irispath = trainpath + str(i).zfill(3) + "_1_" + str(j) + ".bmp"

        eye = cv2.imread(irispath, cv2.IMREAD_GRAYSCALE)
        iris, pupil = IrisLocalization(eye)
        normalized = IrisNormalization(eye, pupil, iris)
        # cv2.imwrite('normalized_iris.png', normalized)
        # ROI = ImageEnhancement(normalized)
        # cv2.imwrite('roi.png', ROI)
        block_size = 16
        iris_code = FeatureExtraction(normalized, block_size)
        train_iris_code[j] = iris_code
        # np.savetxt('iris_code_3.txt', iris_code, fmt='%d')

    for k in range(1,5):
        # Construct the file path for the current training image
        irispath = testpath + str(i).zfill(3) + "_2_" + str(k) + ".bmp"

        eye = cv2.imread(irispath, cv2.IMREAD_GRAYSCALE)
        iris, pupil = IrisLocalization(eye)
        normalized = IrisNormalization(eye, pupil, iris)
        # cv2.imwrite('normalized_iris.png', normalized)
        # ROI = ImageEnhancement(normalized)
        # cv2.imwrite('roi.png', ROI)
        block_size = 16
        iris_code = FeatureExtraction(normalized, block_size)
        test_iris_code[k] = iris_code
    
    count = 0
    for j in range(1, 4):
        for k in range (1, 5):
            train_tmp = train_iris_code[j].astype(int).tolist()
            test_tmp = test_iris_code[k].astype(int).tolist()
            min_distance, best_shift = find_min_hamming_distance(train_tmp, test_tmp)
            if (min_distance <= 0.32):
                count += 1
            
    
    print(f"Test {i}: ", round(count/12 * 100), "%")
    med = med + count

print("overall: ", med / 2 /12 * 100, "%")

# # Example usage
# # Load iris codes from text files
# iris_code_1 = np.loadtxt('D:/at school/2024.1/Biometric Authentication System/Project/Biometric/iris_code_1.txt', dtype=int)
# iris_code_2 = np.loadtxt('D:/at school/2024.1/Biometric Authentication System/Project/Biometric/iris_code_2.txt', dtype=int)
# iris_code_3 = np.loadtxt('D:/at school/2024.1/Biometric Authentication System/Project/Biometric/iris_code_3.txt', dtype=int)

# # Compute Hamming distance
# distance_12 = HammingDistance(iris_code_1, iris_code_2)
# print(f"Hamming Distance 1-2: {distance_12}")
# distance_23 = HammingDistance(iris_code_2, iris_code_3)
# print(f"Hamming Distance 2-3: {distance_23}")
# distance_13 = HammingDistance(iris_code_1, iris_code_3)
# print(f"Hamming Distance 1-3: {distance_13}")

# iris_code_1 = iris_code_1.tolist()
# iris_code_2 = iris_code_2.tolist()
# iris_code_3 = iris_code_3.tolist()

# min_distance, best_shift = find_min_hamming_distance(iris_code_1, iris_code_2)
# print(f"Minimum Hamming Distance 12: {min_distance}")
# print(f"Optimal Shift 12: {best_shift}")

# min_distance, best_shift = find_min_hamming_distance(iris_code_1, iris_code_3)
# print(f"Minimum Hamming Distance 13: {min_distance}")
# print(f"Optimal Shift 13: {best_shift}")

# min_distance, best_shift = find_min_hamming_distance(iris_code_2, iris_code_3)
# print(f"Minimum Hamming Distance 23: {min_distance}")
# print(f"Optimal Shift 23: {best_shift}")

Test 1:  67 %
Test 2:  92 %
overall:  79.16666666666666 %


In [27]:
def Authenticate(iris_code, stored_templates, threshold=0.32):
    for template in stored_templates:
        distance = HammingDistance(iris_code, template)
        if distance < threshold:
            return True  # Authentication successful
    return False  # Authentication failed

In [28]:
# sucess = Authenticate(iris_code_3, [iris_code_1, iris_code_2])
# print(f"Authentication: {sucess}")